In [ ]:
%pip install -q dagshub mlflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.7/733.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import timm
import time
from tqdm import tqdm
import sklearn
import glob
import shutil



In [ ]:
!unzip -q /content/data.zip -d /content/dataset


[/content/data.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/data.zip or
        /content/data.zip.zip, and cannot find /content/data.zip.ZIP, period.


In [ ]:
!ls /content/dataset

ls: cannot access '/content/dataset': No such file or directory


In [ ]:

from sklearn.model_selection import train_test_split

# STEP 1: Setup
original_dataset_dir = "/content/dataset"
output_dir = "/content/split_data"         # Output split folder

classes = ["Bengin cases", "Malignant cases", "Normal cases"]
image_paths = []
labels = []

# STEP 2: Load image paths and class labels
for cls in classes:
    cls_folder = os.path.join(original_dataset_dir, cls)
    for img_path in glob.glob(os.path.join(cls_folder, '*')):
        image_paths.append(img_path)
        labels.append(cls)

print(f"Total images found: {len(image_paths)}")

# STEP 3: Split into Train+Val (80%) and Test (20%)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# STEP 4: Split Train+Val into Train (80%) and Val (20%)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, stratify=y_train_val, random_state=42
)

# Helper function to copy files into target folder
def save_split(split_name, X, y):
    for path, label in zip(X, y):
        split_folder = os.path.join(output_dir, split_name, label)
        os.makedirs(split_folder, exist_ok=True)
        shutil.copy(path, split_folder)

# STEP 5: Copy images to respective folders
save_split("train", X_train, y_train)
save_split("val", X_val, y_val)
save_split("test", X_test, y_test)

# ✅ DONE
print("\n✅ Train/Val/Test split completed and saved to:", output_dir)


Total images found: 2097

✅ Train/Val/Test split completed and saved to: /content/split_data


In [ ]:
import os
from collections import defaultdict

def count_images_per_class(root_dir):
    class_counts = defaultdict(int)

    for class_name in os.listdir(root_dir):
        class_path = os.path.join(root_dir, class_name)
        if os.path.isdir(class_path):
            image_files = [f for f in os.listdir(class_path)
                           if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            class_counts[class_name] = len(image_files)

    return class_counts

# Example usage:
train_dir = "/content/split_data/train"
val_dir = "/content/split_data/val"
test_dir = "/content/split_data/test"

print("Train set:", count_images_per_class(train_dir))
print("Validation set:", count_images_per_class(val_dir))
print("Test set:", count_images_per_class(test_dir))


Train set: defaultdict(<class 'int'>, {'Normal cases': 403, 'Malignant cases': 359, 'Bengin cases': 579})
Validation set: defaultdict(<class 'int'>, {'Normal cases': 101, 'Malignant cases': 90, 'Bengin cases': 145})
Test set: defaultdict(<class 'int'>, {'Normal cases': 127, 'Malignant cases': 112, 'Bengin cases': 181})


In [ ]:
import os
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from collections import defaultdict

# Settings
train_dir = "/content/split_data/train"
target_class_count = 600  # Max class count in training set

# Define augmentation generator
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Count current images
def count_images(root_dir):
    class_counts = defaultdict(int)
    for class_name in os.listdir(root_dir):
        class_folder = os.path.join(root_dir, class_name)
        image_files = [f for f in os.listdir(class_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        class_counts[class_name] = len(image_files)
    return class_counts

class_counts = count_images(train_dir)
print("Before balancing:", class_counts)

# Apply balancing
for class_name, count in class_counts.items():
    if count >= target_class_count:
        continue  # Already balanced

    class_folder = os.path.join(train_dir, class_name)
    image_files = [f for f in os.listdir(class_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    needed = target_class_count - count
    print(f"\n🔄 Augmenting '{class_name}' ({count} → {target_class_count}), need {needed} more")

    i = 0
    while i < needed:
        for img_name in image_files:
            img_path = os.path.join(class_folder, img_name)
            img = tf.keras.preprocessing.image.load_img(img_path)
            x = tf.keras.preprocessing.image.img_to_array(img)
            x = x.reshape((1,) + x.shape)

            prefix = os.path.splitext(img_name)[0]

            for batch in datagen.flow(x, batch_size=1, save_to_dir=class_folder,
                                      save_prefix=f"{prefix}_aug", save_format='jpeg'):
                i += 1
                if i >= needed:
                    break
            if i >= needed:
                break

# Re-count
balanced_counts = count_images(train_dir)
print("\n✅ After balancing:", balanced_counts)


Before balancing: defaultdict(<class 'int'>, {'Normal cases': 506, 'Malignant cases': 359, 'Bengin cases': 579})

🔄 Augmenting 'Normal cases' (506 → 600), need 94 more

🔄 Augmenting 'Malignant cases' (359 → 600), need 241 more

🔄 Augmenting 'Bengin cases' (579 → 600), need 21 more

✅ After balancing: defaultdict(<class 'int'>, {'Normal cases': 598, 'Malignant cases': 597, 'Bengin cases': 600})


In [ ]:
import os
import time
import torch
import mlflow
import mlflow.pytorch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import timm
from tqdm import tqdm

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# Paths
train_dir = '/content/split_data/train'
val_dir = '/content/split_data/val'
test_dir = '/content/split_data/test'

# Hyperparameters
batch_size = 32
num_classes = 3
lr = 3e-4
num_epochs = 15
image_size = 224

# Transforms
train_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])
val_test_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(val_dir, transform=val_test_transform)
test_dataset = datasets.ImageFolder(test_dir, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"Classes: {train_dataset.classes}")

# Load model
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=num_classes)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Training function
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0, 0, 0
    for images, labels in tqdm(loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)
    return running_loss / total, correct / total

# Evaluation function
def eval_epoch(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)
    return running_loss / total, correct / total

# MLflow logging
with mlflow.start_run():
    # Log hyperparameters
    mlflow.log_params({
        "model": "vit_base_patch16_224",
        "image_size": image_size,
        "batch_size": batch_size,
        "learning_rate": lr,
        "num_epochs": num_epochs,
        "num_classes": num_classes,
    })

    best_val_acc = 0
    model_path = "best_vit_lung_cancer_model.pth"
    start_time = time.time()

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, val_acc = eval_epoch(model, val_loader, criterion)

        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        # Save best model and metrics
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_metrics = {
                "train_loss": train_loss,
                "train_acc": train_acc,
                "val_loss": val_loss,
                "val_acc": val_acc
            }
            torch.save(model.state_dict(), model_path)
            print("✅ Saved Best Model")

    total_time = time.time() - start_time
    model_size_mb = os.path.getsize(model_path) / (1024 ** 2)

    # Load best model and test
    print("\n🧪 Testing best model on test set...")
    model.load_state_dict(torch.load(model_path))
    test_loss, test_acc = eval_epoch(model, test_loader, criterion)

    # Final logging
    mlflow.log_metrics({
        **best_metrics,
        "test_loss": test_loss,
        "test_accuracy": test_acc,
        "training_time_sec": total_time,
        "model_size_mb": model_size_mb
    })

    mlflow.log_artifact(model_path)


print("\n✅ Training complete. Best model and metrics logged to MLflow.")



Using device: cuda
Classes: ['Bengin cases', 'Malignant cases', 'Normal cases']

Epoch 1/15


100%|██████████| 57/57 [01:00<00:00,  1.07s/it]


Train Loss: 1.1312 | Train Acc: 0.4730
Val Loss: 0.9992 | Val Acc: 0.4970
✅ Saved Best Model

Epoch 2/15


100%|██████████| 57/57 [01:00<00:00,  1.05s/it]


Train Loss: 0.7877 | Train Acc: 0.6156
Val Loss: 0.7886 | Val Acc: 0.6369
✅ Saved Best Model

Epoch 3/15


...

Train Loss: 0.1965 | Train Acc: 0.9326
Val Loss: 0.4267 | Val Acc: 0.8244

Epoch 14/15


100%|██████████| 57/57 [00:59<00:00,  1.05s/it]


Train Loss: 0.1508 | Train Acc: 0.9476
Val Loss: 0.5957 | Val Acc: 0.8155

Epoch 15/15


100%|██████████| 57/57 [00:59<00:00,  1.05s/it]


Train Loss: 0.1245 | Train Acc: 0.9543
Val Loss: 0.5556 | Val Acc: 0.8423

🧪 Testing best model on test set...
🏃 View run clean-turtle-535 at: https://dagshub.com/Zappy17/research_paper.mlflow/#/experiments/0/runs/d9af4b4701eb4efbb969f996612cd175
🧪 View experiment at: https://dagshub.com/Zappy17/research_paper.mlflow/#/experiments/0

✅ Training complete. Best model and metrics logged to MLflow.


In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# Paths
train_dir = '/content/split_data/train'
val_dir = '/content/split_data/val'
test_dir = '/content/split_data/test'

# Hyperparameters
batch_size = 32
num_classes = 3
lr = 3e-4
num_epochs = 15
image_size = 224

# Transforms
train_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
])
val_test_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
])

# Datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(val_dir, transform=val_test_transform)
test_dataset = datasets.ImageFolder(test_dir, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"Classes: {train_dataset.classes}")

# Load and modify VGG16
model = models.vgg16(pretrained=True)
for param in model.features.parameters():
    param.requires_grad = False  # Freeze conv base
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
model = model.to(device)

# Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Training and evaluation functions
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for images, labels in tqdm(loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return running_loss / total, correct / total

def eval_epoch(model, loader, criterion):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return running_loss / total, correct / total

# Start MLflow logging
with mlflow.start_run():
    mlflow.log_params({
        "model": "vgg16",
        "image_size": image_size,
        "batch_size": batch_size,
        "learning_rate": lr,
        "num_epochs": num_epochs,
        "num_classes": num_classes,
        "frozen_base": True
    })

    best_val_acc = 0
    model_path = "best_vgg16_lung_model.pth"
    start_time = time.time()

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, val_acc = eval_epoch(model, val_loader, criterion)

        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), model_path)
            best_metrics = {
                "train_loss": train_loss,
                "train_acc": train_acc,
                "val_loss": val_loss,
                "val_acc": val_acc
            }
            print("✅ Saved Best Model")

    total_time = time.time() - start_time
    model_size_mb = os.path.getsize(model_path) / (1024 ** 2)

    # Load best model for testing
    model.load_state_dict(torch.load(model_path))
    test_loss, test_acc = eval_epoch(model, test_loader, criterion)

    # Log final best metrics
    mlflow.log_metrics({
        **best_metrics,
        "test_loss": test_loss,
        "test_accuracy": test_acc,
        "training_time_sec": total_time,
        "model_size_mb": model_size_mb
    })

    mlflow.log_artifact(model_path)

    print(f"\n🧪 Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")
    print(f"⏱️ Training Time: {total_time:.2f}s | 💾 Model Size: {model_size_mb:.2f}MB")

print("\n✅ Training complete. Best model and metrics logged to MLflow.")


Using device: cuda
Classes: ['Bengin cases', 'Malignant cases', 'Normal cases']


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 97.5MB/s]



Epoch 1/15


100%|██████████| 57/57 [00:14<00:00,  3.95it/s]


Train Loss: 0.3092 | Train Acc: 0.8975
Val Loss: 0.2108 | Val Acc: 0.9315
✅ Saved Best Model

Epoch 2/15


...

Train Loss: 0.0602 | Train Acc: 0.9861
Val Loss: 0.2470 | Val Acc: 0.9762

Epoch 14/15


100%|██████████| 57/57 [00:13<00:00,  4.22it/s]


Train Loss: 0.0487 | Train Acc: 0.9928
Val Loss: 0.4076 | Val Acc: 0.9613

Epoch 15/15


100%|██████████| 57/57 [00:13<00:00,  4.14it/s]


Train Loss: 0.0498 | Train Acc: 0.9894
Val Loss: 0.2919 | Val Acc: 0.9583

🧪 Test Loss: 0.1219 | Test Accuracy: 0.9714
⏱️ Training Time: 254.54s | 💾 Model Size: 512.22MB
🏃 View run nervous-sponge-591 at: https://dagshub.com/Zappy17/research_paper.mlflow/#/experiments/0/runs/ecf5f72d69904f42897082b7a1b64ef9
🧪 View experiment at: https://dagshub.com/Zappy17/research_paper.mlflow/#/experiments/0

✅ Training complete. Best model and metrics logged to MLflow.


In [ ]:
import os
import time
import torch
import mlflow
import timm
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

# ✅ Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# ✅ Paths
train_dir = '/content/split_data/train'
val_dir = '/content/split_data/val'
test_dir = '/content/split_data/test'

# ✅ Hyperparameters
batch_size = 32
num_classes = 3
lr = 3e-4
num_epochs = 15
image_size = 224

# ✅ Transforms
train_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])
val_test_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])

# ✅ Datasets & loaders
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(val_dir, transform=val_test_transform)
test_dataset = datasets.ImageFolder(test_dir, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"Classes: {train_dataset.classes}")

# ✅ Load ViT model
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=num_classes)

# ❄️ Freeze first 10 transformer blocks
for name, param in model.named_parameters():
    if any(name.startswith(f'blocks.{i}') for i in range(10)):
        param.requires_grad = False
    else:
        param.requires_grad = True

model = model.to(device)

# ✅ Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

# ✅ Train loop
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0, 0, 0
    for images, labels in tqdm(loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return running_loss / total, correct / total

# ✅ Eval loop
def eval_epoch(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return running_loss / total, correct / total

# ✅ MLflow logging for best model
with mlflow.start_run():
    mlflow.log_params({
        "model": "vit_base_patch16_224",
        "fine_tuned_blocks": "blocks.10, blocks.11 + head",
        "image_size": image_size,
        "batch_size": batch_size,
        "learning_rate": lr,
        "num_epochs": num_epochs,
        "num_classes": num_classes,
    })

    best_val_acc = 0
    model_path = "best_vit_frozen_model.pth"
    start_time = time.time()

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, val_acc = eval_epoch(model, val_loader, criterion)

        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), model_path)
            print("✅ Saved Best Model")

    total_time = time.time() - start_time
    print(f"\n⏱️ Total training time: {total_time:.2f} seconds")
    mlflow.log_metric("training_time_sec", total_time)

    # ✅ Model size
    model_size_mb = os.path.getsize(model_path) / (1024 ** 2)
    print(f"💾 Model size: {model_size_mb:.2f} MB")
    mlflow.log_metric("model_size_mb", model_size_mb)

    # ✅ Test best model
    print("\n🧪 Testing best model on test set...")
    model.load_state_dict(torch.load(model_path))
    test_loss, test_acc = eval_epoch(model, test_loader, criterion)
    print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

    mlflow.log_metrics({
        "test_loss": test_loss,
        "test_accuracy": test_acc,
        "best_val_acc": best_val_acc
    })

    # ✅ Log best model
    try:
        mlflow.pytorch.log_model(model, artifact_path="vit_finetuned_best")
    except Exception as e:
        print(f"⚠️ Warning: Failed to log model: {e}")

print("\n✅ Training complete.")


Using device: cuda
Classes: ['Bengin cases', 'Malignant cases', 'Normal cases']



Epoch 1/15


100%|██████████| 57/57 [00:44<00:00,  1.27it/s]


Train Loss: 0.5633 | Train Acc: 0.7850
Val Loss: 0.4000 | Val Acc: 0.8631
✅ Saved Best Model

Epoch 2/15


100%|██████████| 57/57 [00:43<00:00,  1.32it/s]


...

Train Loss: 0.0041 | Train Acc: 0.9994
Val Loss: 0.0661 | Val Acc: 0.9851

Epoch 15/15


100%|██████████| 57/57 [00:43<00:00,  1.31it/s]


Train Loss: 0.0012 | Train Acc: 1.0000
Val Loss: 0.0611 | Val Acc: 0.9851

⏱️ Total training time: 727.09 seconds
💾 Model size: 327.36 MB

🧪 Testing best model on test set...
Test Loss: 0.0694 | Test Accuracy: 0.9762


2025/06/12 23:09:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


⚠️ Warning: Failed to log model: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
🏃 View run whimsical-penguin-55 at: https://dagshub.com/Zappy17/research_paper.mlflow/#/experiments/0/runs/01e934c7a63e4daabc83e86718114985
🧪 View experiment at: https://dagshub.com/Zappy17/research_paper.mlflow/#/experiments/0

✅ Training complete.


In [ ]:
import os
import time
import torch
import mlflow
import timm
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

# ✅ Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# ✅ Dataset paths
train_dir = '/content/split_data/train'
val_dir = '/content/split_data/val'
test_dir = '/content/split_data/test'

# ✅ Hyperparameters
batch_size = 32
num_classes = 3
lr = 3e-4
num_epochs = 15
image_size = 224

# ✅ Transforms
train_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])
val_test_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])

# ✅ Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(val_dir, transform=val_test_transform)
test_dataset = datasets.ImageFolder(test_dir, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"Classes: {train_dataset.classes}")

# ✅ Load pretrained VGG16 model
model = timm.create_model('vgg16', pretrained=True, num_classes=num_classes)

# ❄️ Freeze all feature extractor layers
for name, param in model.named_parameters():
    if "features" in name:
        param.requires_grad = False
    else:
        param.requires_grad = True

model = model.to(device)

# ✅ Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

# ✅ Train one epoch
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0, 0, 0
    for images, labels in tqdm(loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return running_loss / total, correct / total

# ✅ Evaluation
def eval_epoch(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return running_loss / total, correct / total

# ✅ Start MLflow run
with mlflow.start_run():
    mlflow.log_params({
        "model": "vgg16",
        "frozen_layers": "all feature extractor layers",
        "image_size": image_size,
        "batch_size": batch_size,
        "learning_rate": lr,
        "num_epochs": num_epochs,
        "num_classes": num_classes,
    })

    best_val_acc = 0
    model_path = "best_vgg16_model.pth"
    start_time = time.time()

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, val_acc = eval_epoch(model, val_loader, criterion)

        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), model_path)
            print("✅ Saved Best Model")

    total_time = time.time() - start_time
    print(f"\n⏱️ Total training time: {total_time:.2f} seconds")
    mlflow.log_metric("training_time_sec", total_time)

    # ✅ Model size
    model_size_mb = os.path.getsize(model_path) / (1024 ** 2)
    print(f"💾 Model size: {model_size_mb:.2f} MB")
    mlflow.log_metric("model_size_mb", model_size_mb)

    # ✅ Evaluate on test set
    print("\n🧪 Testing best model on test set...")
    model.load_state_dict(torch.load(model_path))
    test_loss, test_acc = eval_epoch(model, test_loader, criterion)
    print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

    mlflow.log_metrics({
        "test_loss": test_loss,
        "test_accuracy": test_acc,
        "best_val_acc": best_val_acc
    })

    # ✅ Try logging model
    try:
        mlflow.pytorch.log_model(model, artifact_path="vgg16_finetuned_best")
    except Exception as e:
        print(f"⚠️ Warning: Could not log model: {e}")

print("\n✅ VGG16 training complete and best model logged.")


Using device: cuda
Classes: ['Bengin cases', 'Malignant cases', 'Normal cases']


model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]


Epoch 1/15


100%|██████████| 57/57 [00:14<00:00,  3.91it/s]


Train Loss: 0.2538 | Train Acc: 0.9103
Val Loss: 0.2566 | Val Acc: 0.9077
✅ Saved Best Model

Epoch 2/15


...

Train Loss: 0.0028 | Train Acc: 0.9989
Val Loss: 0.3071 | Val Acc: 0.9583

Epoch 15/15


100%|██████████| 57/57 [00:14<00:00,  4.03it/s]


Train Loss: 0.0051 | Train Acc: 0.9983
Val Loss: 0.2162 | Val Acc: 0.9792
✅ Saved Best Model

⏱️ Total training time: 289.09 seconds
💾 Model size: 512.22 MB

🧪 Testing best model on test set...
Test Loss: 0.0979 | Test Accuracy: 0.9810


2025/06/12 23:28:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


⚠️ Warning: Could not log model: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
🏃 View run luminous-perch-562 at: https://dagshub.com/Zappy17/research_paper.mlflow/#/experiments/0/runs/e79fb608341045f394a184d736194f50
🧪 View experiment at: https://dagshub.com/Zappy17/research_paper.mlflow/#/experiments/0

✅ VGG16 training complete and best model logged.


In [ ]:
import os
import time
import mlflow
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from transformers import AutoImageProcessor, AutoModelForImageClassification
from tqdm import tqdm

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# Paths
train_dir = '/content/split_data/train'
val_dir = '/content/split_data/val'
test_dir = '/content/split_data/test'

# Hyperparameters
batch_size = 32
num_classes = 3
lr = 3e-4
num_epochs = 15
image_size = 224

# Transforms
train_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])
val_test_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])

# Datasets & Loaders
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(val_dir, transform=val_test_transform)
test_dataset = datasets.ImageFolder(test_dir, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"Classes: {train_dataset.classes}")

# Load COVID ViT pretrained model
model = AutoModelForImageClassification.from_pretrained(
    "DunnBC22/vit-base-patch16-224-in21k_covid_19_ct_scans"
)

# Modify classifier head
model.classifier = nn.Sequential(
    nn.Linear(model.classifier.in_features, num_classes)
)

# Freeze all ViT encoder layers
for name, param in model.named_parameters():
    if not name.startswith("classifier"):
        param.requires_grad = False

model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

# Training & Evaluation loops
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for images, labels in tqdm(loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return running_loss / total, correct / total

def eval_epoch(model, loader, criterion):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return running_loss / total, correct / total

# MLflow logging
with mlflow.start_run():
    mlflow.log_params({
        "model": "covid_vit_transfer_learning",
        "image_size": image_size,
        "batch_size": batch_size,
        "learning_rate": lr,
        "num_epochs": num_epochs,
        "num_classes": num_classes
    })

    best_val_acc = 0
    model_path = "covid_vit_lung_teacher.pth"
    start_time = time.time()

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, val_acc = eval_epoch(model, val_loader, criterion)

        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), model_path)
            print("✅ Saved Best Model")

    total_time = time.time() - start_time
    mlflow.log_metric("training_time_sec", total_time)
    mlflow.log_metric("val_best_accuracy", best_val_acc)

    # Log model size
    model_size = os.path.getsize(model_path) / (1024 ** 2)
    mlflow.log_metric("model_size_mb", model_size)

    # Test evaluation
    model.load_state_dict(torch.load(model_path))
    test_loss, test_acc = eval_epoch(model, test_loader, criterion)
    mlflow.log_metrics({"test_loss": test_loss, "test_accuracy": test_acc})

    # Log model
    try:
        mlflow.pytorch.log_model(model, "covid_vit_teacher")
    except:
        print("⚠️ Could not log model to MLflow (likely due to custom transformers model)")

print("\n✅ Training complete. Teacher ViT model ready.")


In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# Paths
train_dir = '/content/split_data/train'
val_dir = '/content/split_data/val'
test_dir = '/content/split_data/test'

# Hyperparameters
batch_size = 32
num_classes = 3
lr = 3e-4
num_epochs = 15
image_size = 224

# Transforms
train_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])
val_test_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])

# Datasets & Loaders
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(val_dir, transform=val_test_transform)
test_dataset = datasets.ImageFolder(test_dir, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"Classes: {train_dataset.classes}")

# Load COVID ViT pretrained model
model = AutoModelForImageClassification.from_pretrained(
    "DunnBC22/vit-base-patch16-224-in21k_covid_19_ct_scans"
)

# Modify classifier head
model.classifier = nn.Sequential(
    nn.Linear(model.classifier.in_features, num_classes)
)

# Freeze all ViT encoder layers
for name, param in model.named_parameters():
    if not name.startswith("classifier"):
        param.requires_grad = False

model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

# Training & Evaluation loops
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for images, labels in tqdm(loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return running_loss / total, correct / total

def eval_epoch(model, loader, criterion):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return running_loss / total, correct / total

# MLflow logging
with mlflow.start_run():
    mlflow.log_params({
        "model": "covid_vit_transfer_learning",
        "image_size": image_size,
        "batch_size": batch_size,
        "learning_rate": lr,
        "num_epochs": num_epochs,
        "num_classes": num_classes
    })

    best_val_acc = 0
    model_path = "covid_vit_lung_teacher.pth"
    start_time = time.time()

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, val_acc = eval_epoch(model, val_loader, criterion)

        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), model_path)
            print("✅ Saved Best Model")

    total_time = time.time() - start_time
    mlflow.log_metric("training_time_sec", total_time)
    mlflow.log_metric("val_best_accuracy", best_val_acc)

    # Log model size
    model_size = os.path.getsize(model_path) / (1024 ** 2)
    mlflow.log_metric("model_size_mb", model_size)

    # Test evaluation
    model.load_state_dict(torch.load(model_path))
    test_loss, test_acc = eval_epoch(model, test_loader, criterion)
    mlflow.log_metrics({"test_loss": test_loss, "test_accuracy": test_acc})

    # Log model
    try:
        mlflow.pytorch.log_model(model, "covid_vit_teacher")
    except:
        print("⚠️ Could not log model to MLflow (likely due to custom transformers model)")

print("\n✅ Training complete. Teacher ViT model ready.")


In [ ]:

# Assumes the same data, transforms, loaders from the teacher code
# Uses: train_loader, val_loader, test_loader, num_classes, device

# ---------------------
# Define Distillation Loss
# ---------------------
def distillation_loss(student_outputs, teacher_outputs, labels, T=4, alpha=0.7):
    kl_loss = nn.KLDivLoss(reduction='batchmean')
    ce_loss = nn.CrossEntropyLoss()

    # Soften probabilities
    student_probs = nn.functional.log_softmax(student_outputs / T, dim=1)
    teacher_probs = nn.functional.softmax(teacher_outputs / T, dim=1)

    distill = kl_loss(student_probs, teacher_probs) * (T * T)
    student_loss = ce_loss(student_outputs, labels)

    return alpha * student_loss + (1 - alpha) * distill

# ---------------------
# Load trained teacher
# ---------------------
from transformers import AutoModelForImageClassification
teacher_model = AutoModelForImageClassification.from_pretrained(
    "DunnBC22/vit-base-patch16-224-in21k_covid_19_ct_scans"
)
teacher_model.classifier = nn.Sequential(nn.Linear(768, num_classes))
teacher_model.load_state_dict(torch.load("covid_vit_lung_teacher.pth"))
teacher_model.to(device)
teacher_model.eval()

# ---------------------
# Load student (VGG16)
# ---------------------
student_model = models.vgg16(pretrained=True)
for param in student_model.features.parameters():
    param.requires_grad = False
student_model.classifier[6] = nn.Linear(4096, num_classes)
student_model.to(device)

# ---------------------
# Training loop
# ---------------------
optimizer = optim.Adam(student_model.parameters(), lr=3e-4)
num_epochs = 15

with mlflow.start_run():
    mlflow.log_params({
        "student_model": "VGG16",
        "teacher_model": "COVID-ViT",
        "distill_T": 4,
        "alpha": 0.7,
        "batch_size": batch_size,
        "learning_rate": 3e-4,
        "num_epochs": num_epochs,
    })

    best_val_acc = 0
    model_path = "distilled_vgg16.pth"

    for epoch in range(num_epochs):
        student_model.train()
        total_loss, correct, total = 0, 0, 0

        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            student_outputs = student_model(images)
            with torch.no_grad():
                teacher_outputs = teacher_model(images).logits

            loss = distillation_loss(student_outputs, teacher_outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * images.size(0)
            _, preds = torch.max(student_outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        train_loss = total_loss / total

        student_model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = student_model(images)
                loss = nn.CrossEntropyLoss()(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = val_correct / val_total

        print(f"Epoch {epoch+1}: Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(student_model.state_dict(), model_path)
            print("✅ Saved best student model")

    # Final test
    student_model.load_state_dict(torch.load(model_path))
    student_model.eval()
    test_loss, test_correct, test_total = 0, 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = student_model(images)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            test_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            test_correct += (preds == labels).sum().item()
            test_total += labels.size(0)

    test_acc = test_correct / test_total
    mlflow.log_metrics({
        "val_best_accuracy": best_val_acc,
        "test_accuracy": test_acc,
        "test_loss": test_loss / test_total
    })

    mlflow.log_artifact(model_path)

print("\n✅ Knowledge Distillation complete. Student model trained.")
